In [1]:
from common.amigos.dataset import AMIGOSDataset

ds = AMIGOSDataset("../../resources/AMIGOS/sampled/AMIGOS_sampled.csv")
ds.__getitem__(1)

([array([[[21, 36, 24],
          [21, 36, 24],
          [23, 32, 43],
          ...,
          [14, 28, 38],
          [21, 25, 38],
          [19, 23, 36]],
  
         [[20, 35, 23],
          [20, 35, 23],
          [22, 31, 42],
          ...,
          [17, 31, 41],
          [18, 22, 35],
          [22, 26, 39]],
  
         [[18, 30, 21],
          [18, 30, 21],
          [20, 28, 37],
          ...,
          [14, 25, 36],
          [19, 27, 32],
          [25, 33, 38]],
  
         ...,
  
         [[21, 24, 14],
          [21, 24, 14],
          [22, 25, 17],
          ...,
          [22, 32, 29],
          [17, 27, 24],
          [17, 27, 24]],
  
         [[25, 28, 18],
          [25, 28, 18],
          [25, 28, 20],
          ...,
          [22, 32, 29],
          [18, 28, 25],
          [19, 29, 26]],
  
         [[24, 27, 17],
          [24, 27, 17],
          [23, 26, 18],
          ...,
          [22, 32, 29],
          [23, 33, 30],
          [26, 36, 33]]], shape=(

In [10]:
import numpy as np

a = ds.__getitem__(1)
for i in a:
    if i is None:
        print("None")
    else:
        print(i.shape if isinstance(i, np.ndarray) else len(i))

77
2
None
(640, 17)
None


In [24]:
len(a[0])

77

In [26]:
np.array(a[0]).shape

(77, 720, 1280, 3)